# Assignment 1


In [2]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [80]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'#'analogy_alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [81]:

import codecs, scipy.sparse, numpy

#create co-occurrence matrix
def create_cooccurrence_matrix(filename,window_size):
    vocabulary={}
    data=[]
    row=[]
    col=[]
    for sentence in codecs.open(filename,"r","utf-8"):
        sentence=sentence.strip()
        tokens=[token for token in corpus]
        for pos,token in enumerate(tokens):
            i=vocabulary.setdefault(token,len(vocabulary))
            start=max(0,pos-window_size)
            end=min(len(tokens),pos+window_size+1)
            for pos2 in xrange(start,end):
                if pos2==pos: 
                    continue
                j=vocabulary.setdefault(tokens[pos2],len(vocabulary))
                data.append(1.); row.append(i); col.append(j);
    cooccurrence_matrix=sparse.coo_matrix((data,(row,col)))
    return vocabulary,cooccurrence_matrix

#create_cooccurrence_matrix('alice.txt',corpus,window_size)
#print(corpus)
#coo=numpy.zeros((V, V))
#coo=[[0 for x in range(V)] for y in range(V)] 
vocabulary={}
data=[]
row=[]
col=[]
for sentence in corpus:
    for pos,word in enumerate(sentence):
        i=vocabulary.setdefault(word,len(vocabulary))
        start=max(0,pos-window_size)
        end=min(len(sentence),pos+window_size+1)
        for pos2 in range(start,end):
            if pos2==pos: 
                continue
            j=vocabulary.setdefault(sentence[pos2],len(vocabulary))
            data.append(1.); row.append(i); col.append(j);
            
cooccurrence_matrix=scipy.sparse.coo_matrix((data,(row,col)))
print(cooccurrence_matrix.T * cooccurrence_matrix)
print(cooccurrence_matrix.toarray())
#print(cooccurrence_matrix.toarray()[1180][0])



  (0, 1180)	1.0
  (0, 1177)	1.0
  (0, 1165)	1.0
  (0, 1164)	1.0
  (0, 1146)	1.0
  (0, 1142)	1.0
  (0, 1138)	1.0
  (0, 1131)	1.0
  (0, 1130)	1.0
  (0, 1123)	1.0
  (0, 1107)	1.0
  (0, 1106)	1.0
  (0, 1092)	1.0
  (0, 1078)	1.0
  (0, 1070)	1.0
  (0, 1069)	1.0
  (0, 1068)	1.0
  (0, 1057)	1.0
  (0, 1048)	1.0
  (0, 1041)	1.0
  (0, 1040)	1.0
  (0, 1035)	1.0
  (0, 1033)	1.0
  (0, 1028)	1.0
  (0, 1006)	1.0
  :	:
  (1181, 113)	1.0
  (1181, 111)	1.0
  (1181, 102)	1.0
  (1181, 52)	1.0
  (1181, 45)	1.0
  (1181, 44)	1.0
  (1181, 43)	1.0
  (1181, 42)	5.0
  (1181, 38)	1.0
  (1181, 37)	1.0
  (1181, 36)	2.0
  (1181, 35)	1.0
  (1181, 34)	1.0
  (1181, 33)	1.0
  (1181, 27)	2.0
  (1181, 26)	1.0
  (1181, 24)	1.0
  (1181, 23)	1.0
  (1181, 20)	4.0
  (1181, 18)	1.0
  (1181, 11)	1.0
  (1181, 7)	1.0
  (1181, 6)	1.0
  (1181, 3)	1.0
  (1181, 1)	1.0
[[ 0.  1.  1. ...  0.  0.  0.]
 [ 1. 16.  3. ...  0.  0.  0.]
 [ 1.  3. 12. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0. 

In [123]:
#find cosine similarity to Alice, Dinah and Rabbit
#print(corpus)

#from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf_vectorizer = TfidfVectorizer()
#tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
#print(tfidf_matrix.shape)
#from gensim.models import TfidfModel


print(tokenizer.word_index['alice'])  #11
print(tokenizer.word_index['dinah'])  #87
print(tokenizer.word_index['rabbit']) #63



cooc=cooccurrence_matrix.toarray()
tf_cooc=numpy.zeros((V, V))
for i, sentence in enumerate(cooc):
    sum=0
    for wordf in sentence:
        sum+=wordf
    for j,wordf in enumerate(sentence):
        if(sum>0):
            tf_cooc[i][j]=cooc[i][j]/sum
        else:
            continue
#print(tf_cooc)

#print(len(cooc))
#print(len(cooc[10]))
#print(V)
#print(cooc[0])
#print(tokenizer.word_index)

from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(tf_cooc[11:12], tf_cooc[87:88]))
print(cosine_similarity(tf_cooc[11:12], tf_cooc[63:64]))
print(cosine_similarity(tf_cooc[87:88], tf_cooc[63:64]))

11
87
63
[[0.45392756]]
[[0.11704115]]
[[0.13608276]]


In [ ]:
#find the closest words to Alice


Discussion of the drawbacks:

In [ ]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [ ]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow


In [242]:
#create CBOW model


In [243]:
#define loss function


In [1]:
#train model


In [ ]:
#prepare data for Skipgram


In [ ]:
#create Skipgram model


In [ ]:
#define loss function for Skipgram


In [ ]:
#train Skipgram model


In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [ ]:
#create Skipgram with additional dense layer


In [ ]:
#define loss function for Skipgram + dense


In [ ]:
#train model for Skipgram + dense


In [ ]:
#Implement your own analogy function


Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings